In [20]:
%pip install Flask joblib numpy pandas nest-asyncio

Note: you may need to restart the kernel to use updated packages.


In [45]:
import pandas as pd
import numpy as np
import joblib
from flask import Flask, request, jsonify
import nest_asyncio

In [46]:
MODEL_FILE_PATH = './random_forest_model.pkl'

EXPECTED_FEATURES = [
    'default_profile_image',
    'statuses_count',
    'followers_count',
    'friends_count',
    'protected',
    'name_length',
    'follower_ratio'
]

In [47]:
try:
    MODEL = joblib.load(MODEL_FILE_PATH)
    print(f"Model {MODEL_FILE_PATH} loaded successfully.")
except FileNotFoundError:
    print(f"FATAL ERROR: Model file not found at {MODEL_FILE_PATH}.")
    raise Exception("Model loading failed.")

Model ./random_forest_model.pkl loaded successfully.


In [48]:
app = Flask(__name__)
@app.route('/predict', methods=['POST'])
def predict():
    """Receives JSON data, runs the prediction, and returns the result."""
    try:
        data = request.get_json(force=True)
        feature_values = [data[f] for f in EXPECTED_FEATURES]
        final_features = np.array(feature_values).reshape(1, -1)

        prediction_class = MODEL.predict(final_features)[0]
        prediction_proba = MODEL.predict_proba(final_features)[0][1] 

        result = {
            'prediction_class': int(prediction_class),
            'suspicion_score_percent': round(prediction_proba * 100, 2),
            'status': 'FAKE PROFILE DETECTED' if prediction_class == 1 else 'GENUINE PROFILE'
        }
        return jsonify(result)

    except KeyError as e:
        return jsonify({'error': 'Missing Feature', 'details': f"Input is missing the feature: {e}"}), 400
    except Exception as e:
        return jsonify({'error': 'Prediction failed', 'details': str(e)}), 500

In [49]:
import requests
import json
import threading
import time

In [50]:
# ------- Starting the Server -------

nest_asyncio.apply()

def run_flask_app():
    app.run(debug=True, use_reloader=False, port=5000)

threading.Thread(target=run_flask_app).start()

print("Flask server started on port 5000. Waiting 3 seconds before testing...")
time.sleep(3)

Flask server started on port 5000. Waiting 3 seconds before testing...
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [ ]:
test_data = {
    "default_profile_image": 1.0,
    "statuses_count": 5,
    "followers_count": 10,
    "friends_count": 5000,
    "protected": 0.0,
    "name_length": 6,              
    "follower_ratio": 0.002
}

try:
    response = requests.post(
        'http://127.0.0.1:5000/predict',
        json=test_data,
        timeout=5
    )

    print(f"\n--- API Test Result (Status: {response.status_code}) ---")
    print(json.dumps(response.json(), indent=4))

    if response.status_code == 200 and response.json().get('prediction_class') == 1:
        print("\nSUCCESS: The API is live and correctly classified the test profile is FAKE.")
    else:
        print("\nFAILURE: Check the feature names and data type in the JSON body.")

except requests.exceptions.ConnectionError:
    print("\nERROR: Could not connect to the Flask Server.")
except Exception as e:
    print(f"\n An unexpected error occurred during testing: {e}")

c:\Users\shukl\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\shukl\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
127.0.0.1 - - [02/Oct/2025 16:16:38] "POST /predict HTTP/1.1" 200 -



--- API Test Result (Status: 200) ---
{
    "prediction_class": 1,
    "status": "FAKE PROFILE DETECTED",
    "suspicion_score_percent": 77.0
}

SUCCESS: The API is live and correctly classified the test profile is FAKE.


127.0.0.1 - - [02/Oct/2025 16:16:45] "GET / HTTP/1.1" 404 -
